In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Layer, Lambda

In [2]:
forestfires = pd.read_csv('D:\\excler\\Assignment\\Neural Networks\\forestfires.csv')
forestfires.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


In [3]:
forestfires.drop(["month", "day"], axis = 1, inplace = True)

In [4]:
forestfires["area"] = np.where(forestfires["area"] > 50, 1, 0)

In [5]:
forestfires["area"].value_counts()
forestfires.isnull().sum()
forestfires.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,...,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,0.046422,0.164410,...,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,0.210600,0.371006,...,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
def norm_func(i):
     x = (i - i.min()) / (i.max() - i.min())
     return (x)

In [7]:
predictors = forestfires.iloc[ :, 0:8]
target = forestfires.iloc[ :, 8]

predictors1 = norm_func(predictors)

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(predictors1, target, test_size = 0.2, stratify = target)

In [9]:
def prep_model(hidden_dim):
    model = Sequential()
    for i in range(1, len(hidden_dim) - 1):
        if (i == 1):
            model.add(Dense(hidden_dim[i], input_dim = hidden_dim[0], activation = "relu"))
        else:
            model.add(Dense(hidden_dim[i], activation = "relu"))
    model.add(Dense(hidden_dim[-1], kernel_initializer = "normal", activation = "sigmoid"))
    model.compile(loss = "binary_crossentropy", optimizer = "rmsprop", metrics = ["accuracy"])
    return model  

In [10]:
first_model = prep_model([8, 50, 40, 20, 1])
first_model.fit(np.array(x_train), np.array(y_train), epochs = 750)
pred_train = first_model.predict(np.array(x_train))

Epoch 1/750
13/13 [==============================] - 1s 3ms/step - loss: 0.6392 - accuracy: 0.9516
Epoch 2/750
13/13 [==============================] - 0s 3ms/step - loss: 0.4841 - accuracy: 0.9540
Epoch 3/750
13/13 [==============================] - 0s 3ms/step - loss: 0.3060 - accuracy: 0.9540
Epoch 4/750
13/13 [==============================] - 0s 3ms/step - loss: 0.2131 - accuracy: 0.9540
Epoch 5/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1925 - accuracy: 0.9540
Epoch 6/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1887 - accuracy: 0.9540
Epoch 7/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1890 - accuracy: 0.9540
Epoch 8/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1896 - accuracy: 0.9540
Epoch 9/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1868 - accuracy: 0.9540
Epoch 10/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1860 - accuracy: 0.9540
Epoch 11/

13/13 [==============================] - 0s 3ms/step - loss: 0.1662 - accuracy: 0.9540
Epoch 84/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1669 - accuracy: 0.9540
Epoch 85/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1668 - accuracy: 0.9540
Epoch 86/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1666 - accuracy: 0.9540
Epoch 87/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1669 - accuracy: 0.9540
Epoch 88/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1665 - accuracy: 0.9540
Epoch 89/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1665 - accuracy: 0.9540
Epoch 90/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1667 - accuracy: 0.9540
Epoch 91/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1657 - accuracy: 0.9540
Epoch 92/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1677 - accuracy: 0.9540
Epoch 93/750


13/13 [==============================] - 0s 2ms/step - loss: 0.1452 - accuracy: 0.9540
Epoch 165/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1469 - accuracy: 0.9540
Epoch 166/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1444 - accuracy: 0.9540
Epoch 167/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1467 - accuracy: 0.9540
Epoch 168/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1446 - accuracy: 0.9540
Epoch 169/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1467 - accuracy: 0.9540
Epoch 170/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1443 - accuracy: 0.9540
Epoch 171/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1435 - accuracy: 0.9540
Epoch 172/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1466 - accuracy: 0.9540
Epoch 173/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1433 - accuracy: 0.9540
Epoc

13/13 [==============================] - 0s 2ms/step - loss: 0.1322 - accuracy: 0.9564
Epoch 246/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1291 - accuracy: 0.9540
Epoch 247/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1344 - accuracy: 0.9564
Epoch 248/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1316 - accuracy: 0.9564
Epoch 249/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1309 - accuracy: 0.9564
Epoch 250/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1282 - accuracy: 0.9588
Epoch 251/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1287 - accuracy: 0.9564
Epoch 252/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1345 - accuracy: 0.9564
Epoch 253/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1288 - accuracy: 0.9564
Epoch 254/750
13/13 [==============================] - 0s 2ms/step - loss: 0.1288 - accuracy: 0.9564
Epoc

13/13 [==============================] - 0s 3ms/step - loss: 0.1232 - accuracy: 0.9564
Epoch 327/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1120 - accuracy: 0.9613
Epoch 328/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1141 - accuracy: 0.9588
Epoch 329/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1091 - accuracy: 0.9588
Epoch 330/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1133 - accuracy: 0.9564
Epoch 331/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1133 - accuracy: 0.9588
Epoch 332/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1107 - accuracy: 0.9588
Epoch 333/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1135 - accuracy: 0.9613
Epoch 334/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1139 - accuracy: 0.9613
Epoch 335/750
13/13 [==============================] - 0s 3ms/step - loss: 0.1113 - accuracy: 0.9613
Epoc

13/13 [==============================] - 0s 2ms/step - loss: 0.0985 - accuracy: 0.9661
Epoch 408/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0935 - accuracy: 0.9613
Epoch 409/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0923 - accuracy: 0.9588
Epoch 410/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0948 - accuracy: 0.9613
Epoch 411/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0958 - accuracy: 0.9588
Epoch 412/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0930 - accuracy: 0.9637
Epoch 413/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0928 - accuracy: 0.9637
Epoch 414/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0933 - accuracy: 0.9564
Epoch 415/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0924 - accuracy: 0.9637
Epoch 416/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0876 - accuracy: 0.9613
Epoc

13/13 [==============================] - 0s 2ms/step - loss: 0.0813 - accuracy: 0.9637
Epoch 489/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0827 - accuracy: 0.9637
Epoch 490/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0799 - accuracy: 0.9709
Epoch 491/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0738 - accuracy: 0.9709
Epoch 492/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0863 - accuracy: 0.9685
Epoch 493/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0949 - accuracy: 0.9613
Epoch 494/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0775 - accuracy: 0.9685
Epoch 495/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0862 - accuracy: 0.9685
Epoch 496/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0778 - accuracy: 0.9685
Epoch 497/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0745 - accuracy: 0.9758
Epoc

13/13 [==============================] - 0s 2ms/step - loss: 0.0897 - accuracy: 0.9685
Epoch 570/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0686 - accuracy: 0.9685
Epoch 571/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0807 - accuracy: 0.9685
Epoch 572/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0673 - accuracy: 0.9685
Epoch 573/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0707 - accuracy: 0.9661
Epoch 574/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0714 - accuracy: 0.9734
Epoch 575/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0646 - accuracy: 0.9734
Epoch 576/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0756 - accuracy: 0.9613
Epoch 577/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0728 - accuracy: 0.9661
Epoch 578/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0774 - accuracy: 0.9661
Epoc

13/13 [==============================] - 0s 3ms/step - loss: 0.0698 - accuracy: 0.9685
Epoch 651/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0601 - accuracy: 0.9709
Epoch 652/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0607 - accuracy: 0.9709
Epoch 653/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0698 - accuracy: 0.9709
Epoch 654/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0703 - accuracy: 0.9831
Epoch 655/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0586 - accuracy: 0.9734
Epoch 656/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0607 - accuracy: 0.9709
Epoch 657/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0735 - accuracy: 0.9734
Epoch 658/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0537 - accuracy: 0.9758
Epoch 659/750
13/13 [==============================] - 0s 2ms/step - loss: 0.0662 - accuracy: 0.9806
Epoc

13/13 [==============================] - 0s 3ms/step - loss: 0.0603 - accuracy: 0.9758
Epoch 732/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0687 - accuracy: 0.9758
Epoch 733/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0798 - accuracy: 0.9709
Epoch 734/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0517 - accuracy: 0.9806
Epoch 735/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0531 - accuracy: 0.9782
Epoch 736/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0584 - accuracy: 0.9709
Epoch 737/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0493 - accuracy: 0.9806
Epoch 738/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0676 - accuracy: 0.9709
Epoch 739/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0547 - accuracy: 0.9806
Epoch 740/750
13/13 [==============================] - 0s 3ms/step - loss: 0.0748 - accuracy: 0.9758
Epoc

In [11]:
pred_train = pd.Series([i[0] for i in pred_train])

size = ["small", "large"]
pred_train_class = pd.Series(["small"]*413)
pred_train_class[[i > 0.5 for i in pred_train]] = "large"

In [12]:
train = pd.concat([x_train, y_train], axis = 1)
train["area"].value_counts()

0    394
1     19
Name: area, dtype: int64

In [13]:
from sklearn.metrics import confusion_matrix
train["original_class"] = "small"
train.loc[train["area"] == 1, "original_class"] = "large"
train.original_class.value_counts()
confusion_matrix(pred_train_class, train["original_class"])
np.mean(pred_train_class == pd.Series(train["original_class"]).reset_index(drop = True)) 
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop = True))

original_class,large,small
row_0,,
large,9,1
small,10,393


In [14]:
pred_test = first_model.predict(np.array(x_test))
pred_test = pd.Series([i[0] for i in pred_test])
pred_test_class = pd.Series(["small"]*104)
pred_test_class[[i>0.5 for i in pred_test]] = "large"
test =pd.concat([x_test, y_test], axis = 1)
test["original_class"] = "small"
test.loc[test["area"] == 1, "original_class"] = "large"

In [15]:
test["original_class"].value_counts()

small    99
large     5
Name: original_class, dtype: int64

In [16]:
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop = True)) 
confusion_matrix(pred_test_class,test["original_class"])
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop = True))

original_class,large,small
row_0,,
small,5,99
